In [1]:
from sympy import *
__VERSION__ = "2.2"
__DATE__ = "2/Nov/2022"

# Carbon Calculations

Various calculations related to the Carbon whitepaper.

In [2]:
def isolate(eqn, var,ix=0):
    "isolates the variable var in the equation eqn and returns result as equation"
    return Eq(var, solve(eqn, var)[ix])

## Core Formulas

In [3]:
x,x0,xint,dx,Dx = symbols("x x_0 x_{int} dx \Delta{x}")
y,y0,yint,dy,Dy = symbols("y y_0 y_{int} dy \Delta{y}")
P,Pmarg,Px,Py,Q,Gam,w = symbols("P_0 P_{marg} P_x P_y Q \Gamma w")                                            

This is the **Pool Invariant Equation** in its canonic **risk asset** form using $P, x_0, \Gamma$

In [4]:
PIE = Eq(
    y,
    x0*P*(x*(Gam - 1) - x0*(Gam - 2))/(Gam*x - x0*(Gam - 1))
)
PIE

Eq(y, P_0*x_0*(x*(\Gamma - 1) - x_0*(\Gamma - 2))/(\Gamma*x - x_0*(\Gamma - 1)))

In [5]:
PINE = PIE.subs(x0, y0/P).simplify()
PINE

Eq(y, y_0*(P_0*x*(\Gamma - 1) - y_0*(\Gamma - 2))/(P_0*\Gamma*x - y_0*(\Gamma - 1)))

This is the **Q-form** of the **Pool Invariant Equation**

In [6]:
PIQE = Eq(
    Q,
    x*y / ( (x-xint)*(y-yint) )
)
PIQE

Eq(Q, x*y/((x - x_{int})*(y - y_{int})))

In [7]:
_m = (x-xint)*(y-yint)
PIQ2E = Eq(PIQE.lhs*_m, PIQE.rhs*_m)
PIQ2E

Eq(Q*(x - x_{int})*(y - y_{int}), x*y)

This is the **Definition Equations** for $Q$ in terms of $\Gamma$ and its inverse

In [8]:
QGDE = Eq(Q,(1-Gam)**2)
QGDE

Eq(Q, (1 - \Gamma)**2)

In [9]:
GQE = isolate(QGDE, Gam)
GQE

Eq(\Gamma, 1 - sqrt(Q))

This is a useful identity between a certain expression of $\Gamma$ and a certain expression of $Q$.

In [10]:
GFQE = Eq((2-Gam)/(1-Gam),(1+1/sqrt(Q)))
GFQE

Eq((2 - \Gamma)/(1 - \Gamma), 1 + 1/sqrt(Q))

In [11]:
(GFQE.lhs-GFQE.rhs).subs(Gam, 1-sqrt(Q)).simplify()

0

## Other Parametrizations

Those are the **Definition Equations** for $x_{int}, y_{int}, P_x, P_y$

In [12]:
XiDE = Eq(xint, x0*(2-Gam)/(1-Gam))
XiDE

Eq(x_{int}, x_0*(2 - \Gamma)/(1 - \Gamma))

In [13]:
XiQE = XiDE.subs(GFQE.lhs, GFQE.rhs)
XiQE

Eq(x_{int}, x_0*(1 + 1/sqrt(Q)))

In [14]:
YiDE = Eq(yint, y0*(2-Gam)/(1-Gam))
YiDE

Eq(y_{int}, y_0*(2 - \Gamma)/(1 - \Gamma))

In [15]:
YiQE = YiDE.subs(GFQE.lhs, GFQE.rhs)
YiQE

Eq(y_{int}, y_0*(1 + 1/sqrt(Q)))

In [16]:
PxDE = Eq(Px, P*Q)
PxDE

Eq(P_x, P_0*Q)

In [17]:
PyDE = Eq(Py, P/Q)
PyDE

Eq(P_y, P_0/Q)

Here are a few other important relationships. Firstly, $P_0$ is the ratio the reference coordinates

In [18]:
PXYE = Eq(P, y0/x0)
PXYE

Eq(P_0, y_0/x_0)

and more importantly also of the intercepts

In [19]:
PXYiE = Eq(P, yint/xint)
PXYiE

Eq(P_0, y_{int}/x_{int})

$Q$ is related to the ratio of the intercept prices

In [20]:
QPE = Eq(Q, sqrt(Py/Px))
QPE

Eq(Q, sqrt(P_y/P_x))

In fact, if we define $w$ as the (percentage) **range width**

In [21]:
WDE = Eq(w, QPE.rhs**2)
WDE

Eq(w, P_y/P_x)

then that width is simply Q squared

In [22]:
WE = Eq(w, QPE.lhs**2)
WE

Eq(w, Q**2)

Also the reference price is the geometric average of the intercept prices

In [23]:
PPE = Eq(P,sqrt(Px*Py))
PPE

Eq(P_0, sqrt(P_x*P_y))

Below formulas are important for actually setting ranges for unidirectional trading where the parameters we have are the amount of risk asset $x_{int}$ as well as the two ends of the range $P_x, P_y$. Our goal is to solve for the parameterset $Q, x_{int}, y_{int}$ that allows us to use the invariant equation `PIQE`. $x_{int}$ is already known, and $Q$ we can get from `QPE`, so what is left is to calculate $y_{int}$.

We first recall a number of definitions

In [24]:
QPE

Eq(Q, sqrt(P_y/P_x))

In [25]:
PPE

Eq(P_0, sqrt(P_x*P_y))

In [26]:
PXYiE

Eq(P_0, y_{int}/x_{int})

and now make a number of new ones

In [27]:
YiPE = Eq(yint, solve(PXYiE, yint)[0].subs(P, PPE.rhs))
YiPE

Eq(y_{int}, x_{int}*sqrt(P_x*P_y))

In [28]:
XiPE = Eq(xint, solve(YiPE, xint)[0])
XiPE

Eq(x_{int}, y_{int}/sqrt(P_x*P_y))

This is the **Swap Equation** derived from it, describing an actual transaction

In [29]:
SE = Eq(
    - Dy,
    P*x0**2*Dx/((Gam*(x - x0) + x0)*(Gam*(x - x0 + Dx) + x0))
)
SE

Eq(-\Delta{y}, P_0*\Delta{x}*x_0**2/((\Gamma*(x - x_0) + x_0)*(\Gamma*(\Delta{x} + x - x_0) + x_0)))

This is the **Marginal Swap Equation** describing an infinitesimal transaction, rearranged with $dx$ on the LHS 

In [30]:
MSE = Eq(
    -dy/dx,
    P*x0**2/(Gam*(x - x0) + x0)**2
)
MSE

Eq(-dy/dx, P_0*x_0**2/(\Gamma*(x - x_0) + x_0)**2)

This is the **Marginal Price Equation** which is the same as the Marginal Swap Equation, just recognising that $-\frac{dy}{dx}=P_{marg}%$

In [31]:
MPE = Eq(Pmarg, MSE.rhs)
MPE

Eq(P_{marg}, P_0*x_0**2/(\Gamma*(x - x_0) + x_0)**2)

This is the **Reverse Marginal Price Equation** with $y,y_0$ instead of $x,x_0$, but with the price parameters unchanged

In [32]:
MPERev0 = MPE.subs(Pmarg, 1/Pmarg).subs(P,1/P).subs(x0,y0).subs(x,y)
MPERev0

Eq(1/P_{marg}, y_0**2/(P_0*(\Gamma*(y - y_0) + y_0)**2))

The is again the **Reverse Marginal Price Equation** but inversed to isolate $P_{marg}$

In [33]:
MPERev = Eq(Pmarg, 1/MPERev0.rhs)
MPERev

Eq(P_{marg}, P_0*(\Gamma*(y - y_0) + y_0)**2/y_0**2)

Now we reparameterize the MPE as a function of $Q, x_{int}$ and of $Q, y_{int}$

In [34]:
MPQXiE = MPE.subs(GQE.lhs,GQE.rhs).subs(x0, solve(XiQE, x0)[0]).simplify()
MPQXiE

Eq(P_{marg}, P_0*Q*x_{int}**2/(-Q*x + Q*x_{int} + x)**2)

In [35]:
MPQYiE = MPERev.subs(GQE.lhs,GQE.rhs).subs(y0, solve(YiQE, y0)[0]).simplify()
MPQYiE

Eq(P_{marg}, P_0*(-Q*y + Q*y_{int} + y)**2/(Q*y_{int}**2))

### Rho and r parametrizations

In [36]:
r, rho =symbols("r \\rho") 
RXE = Eq(x0, r*x)
RXIE = Eq(x, rho*x0)
RXE

Eq(x_0, r*x)

In [37]:
RXIE

Eq(x, \rho*x_0)

In [38]:
MPRE = MPE.subs(RXE.lhs, RXE.rhs).simplify()
MPRE

Eq(P_{marg}, P_0*r**2/(\Gamma*(r - 1) - r)**2)

In [39]:
MPRIE = MPE.subs(RXIE.lhs, RXIE.rhs).simplify()
MPRIE

Eq(P_{marg}, P_0/(\Gamma*(\rho - 1) + 1)**2)

For completeness, here also the invariant and swap equations using $r,\rho$

In [40]:
PIRE = PIE.subs(RXE.lhs, RXE.rhs).simplify()
PIRE

Eq(y, -P_0*r*x*(-\Gamma + r*(\Gamma - 2) + 1)/(\Gamma - r*(\Gamma - 1)))

In [41]:
PIRIE = PIE.subs(RXIE.lhs, RXIE.rhs).simplify()
PIRIE

Eq(y, P_0*x_0*(-\Gamma + \rho*(\Gamma - 1) + 2)/(\Gamma*\rho - \Gamma + 1))

In [42]:
SRE = SE.subs(RXE.lhs, RXE.rhs).simplify()
SRE

Eq(\Delta{y}, P_0*\Delta{x}*r**2*x/((\Gamma*(r - 1) - r)*(\Gamma*(\Delta{x} - r*x + x) + r*x)))

In [43]:
SRIE = Eq(SE.lhs, P*Dx*x0 / ((Gam*(rho-1)+1)*(Gam*(Dx+x0*(rho-1))+x0)))
SRIE

Eq(-\Delta{y}, P_0*\Delta{x}*x_0/((\Gamma*(\Delta{x} + x_0*(\rho - 1)) + x_0)*(\Gamma*(\rho - 1) + 1)))

In [44]:
(SRIE.rhs - SE.rhs.subs(RXIE.lhs, RXIE.rhs)).simplify()

0

### Adjusting active ranges

As a reminder, and _inactive_ range is a range where the current market price is outside the range, and an _active_ range is one where it is inside. Active ranges split into _active active_ and _inactive active_ ranges. In the former, the market price equals the marginal price of the curve, in the latter the marginal price is somewhat off (in the direction where the marginal price shown in the direction of the AMM is _worse_ than that of the market, so it won't trade.

An active active AMM whose curve is to be readjusted to remain active active has only one remaining degree of freedom -- one can choose either width or mid price, but not both. One can of course change the parameters such that it turns inactive active, in which case the parameters space is two dimensional (width and mid-price), but there is a non-trivial boundary condition to be taken into account because one is starting from the boundary. When adjusting an inactive inactive range, the space is locally fully 2-dimensional, but at one point the boundary must be taken into account.

In a first step, we express our marginal price as a function of $P_0=\sqrt{P_xP_y}$ and $w=P_y/P_x$

In [45]:
MPWYiE = MPQYiE.subs(Q, -solve(WE, Q)[0])
MPWYiE

Eq(P_{marg}, P_0*(-sqrt(w)*y + sqrt(w)*y_{int} + y)**2/(sqrt(w)*y_{int}**2))

We then express $P_0$ as a function of $w$ (and the other paramters), defining the 1-dimensional curve in $P_0, w$ space that any active active reparamterization must fulfil. 

In [46]:
P0WYiE = Eq(P, solve(MPWYiE, P)[0])
P0WYiE

Eq(P_0, P_{marg}*sqrt(w)*y_{int}**2/(-sqrt(w)*y + sqrt(w)*y_{int} + y)**2)

Uncomment the below to obtain the closed form solution for the inverse equation $w=w(P_0)$

In [47]:
#WP0YiE = Eq(w, solve(P0WYiE,w)[0].simplify())
#WP0YiE

For the inequality, we first note that $P_0\propto P_{marg}$. We now need to distinguish the following two cases (note that the price is always quoted as numeraire / risk asset) that ensure that the AMM either matches the market price, or it is _out of the money_, ie not willing to trade at prevailing prices.

1. **AMM sells risk asset.** If the AMM sells the risk asset it is only willing to sell it at market _or higher_, ie $P_{marg} \geq P_{market}$

1. **AMM buys risk asset.** If the AMM buys the risk asset it is only willing to buy it at market _or lower_, ie $P_{marg} \leq P_{market}$

## Creating the Formulas object for export

The `FORMULAS` object contains all key formulas of this worksheet, and it allows to easily import and use them in other workbooks. 

In [48]:
try:
    from .formulalib import Formulas, __VERSION__ as _fversion, __DATE__ as _fdate
except:
    from formulalib import Formulas, __VERSION__ as _fversion, __DATE__ as _fdate
print(f"formulalib Version v{_fversion} ({_fdate})")
FORMULAS = Formulas()

formulalib Version v2.1.2 (17/Nov/2022)


### Core equations

In [49]:
FORMULAS.add(
    "PIE", PIE, 
    "Pool Invariant Equation (risk asset)", 
    "expressed as y as a function of x and parameters P, x0, Gamma", 
)

Eq(y, P_0*x_0*(x*(\Gamma - 1) - x_0*(\Gamma - 2))/(\Gamma*x - x_0*(\Gamma - 1)))

In [50]:
FORMULAS.add(
    "PINE", PINE, 
    "Pool Invariant Equation (numeraire)", 
    "expressed as y as a function of x and parameters P, y0, Gamma", 
)

Eq(y, y_0*(P_0*x*(\Gamma - 1) - y_0*(\Gamma - 2))/(P_0*\Gamma*x - y_0*(\Gamma - 1)))

In [51]:
FORMULAS.add(
    "PIQE", PIQE, 
    "Pool Invariant Equation (Q-form)", 
    "expressed as Q equals function of x,y with the intercepts as parameters", 
)

Eq(Q, x*y/((x - x_{int})*(y - y_{int})))

In [52]:
FORMULAS.add(
    "PIQ2E", PIQ2E, 
    "Pool Invariant Equation (Q-form)", 
    "expressed as Q equals function of x,y with the intercepts as parameters (alternitive form)", 
)

Eq(Q*(x - x_{int})*(y - y_{int}), x*y)

In [53]:
FORMULAS.add(
    "PIRE", PIRE, 
    "Pool Invariant Equation (r)", 
    "expressed as y as a function of r and parameters P, x0, Gamma", 
)

Eq(y, -P_0*r*x*(-\Gamma + r*(\Gamma - 2) + 1)/(\Gamma - r*(\Gamma - 1)))

In [54]:
FORMULAS.add(
    "PIRIE", PIRIE, 
    "Pool Invariant Equation (rho)", 
    "expressed as y as a function of rho and parameters P, x0, Gamma", 
)

Eq(y, P_0*x_0*(-\Gamma + \rho*(\Gamma - 1) + 2)/(\Gamma*\rho - \Gamma + 1))

In [55]:
FORMULAS.add(
    "SE", SE, 
    "Swap Equation", 
    "expressed as minus Delta y as a function of Delta x and all other parameters and variables from the PIE", 
)

Eq(-\Delta{y}, P_0*\Delta{x}*x_0**2/((\Gamma*(x - x_0) + x_0)*(\Gamma*(\Delta{x} + x - x_0) + x_0)))

In [56]:
FORMULAS.add(
    "SRE", SRE, 
    "Swap Equation (r)", 
    "expressed as minus Delta y as a function of Delta x and all other parameters and variables from the PIRE", 
)

Eq(\Delta{y}, P_0*\Delta{x}*r**2*x/((\Gamma*(r - 1) - r)*(\Gamma*(\Delta{x} - r*x + x) + r*x)))

In [57]:
FORMULAS.add(
    "SRIE", SRIE, 
    "Swap Equation (rho)", 
    "expressed as minus Delta y as a function of Delta x and all other parameters and variables from the PIRIE", 
)

Eq(-\Delta{y}, P_0*\Delta{x}*x_0/((\Gamma*(\Delta{x} + x_0*(\rho - 1)) + x_0)*(\Gamma*(\rho - 1) + 1)))

In [58]:
FORMULAS.add(
    "MSE", MSE, 
    "Marginal Swap Equation", 
    "the Swap Equation for Delta x -> dx; dx is brought to the LHS", 
)

Eq(-dy/dx, P_0*x_0**2/(\Gamma*(x - x_0) + x_0)**2)

In [59]:
FORMULAS.add(
    "MPE", MPE, 
    "Marginal Price Equation", 
    "this is the same as the Marginal Swap Equation, except that the LHS is recognized as the marginal price", 
)

Eq(P_{marg}, P_0*x_0**2/(\Gamma*(x - x_0) + x_0)**2)

In [60]:
FORMULAS.add(
    "MPERev", MPERev, 
    "Reverse Marginal Price Equation", 
    "Marginal price eqaution seen from the other side (ie based on y, y0)", 
)

Eq(P_{marg}, P_0*(\Gamma*(y - y_0) + y_0)**2/y_0**2)

In [61]:
FORMULAS.add(
    "MPQXiE", MPQXiE, 
    "Marginal Price Equation (Q,xint)", 
    "Reparametrization of the MPE as a function of $Q, x_{int}$", 
)

Eq(P_{marg}, P_0*Q*x_{int}**2/(-Q*x + Q*x_{int} + x)**2)

In [62]:
FORMULAS.add(
    "MPQYiE", MPQYiE, 
    "Marginal Price Equation (Q,yint)", 
    "Reparametrization of the MPE as a function of $Q, y_{int}$", 
)

Eq(P_{marg}, P_0*(-Q*y + Q*y_{int} + y)**2/(Q*y_{int}**2))

In [63]:
FORMULAS.add(
    "MPWYiE", MPWYiE, 
    "Marginal Price Equation (w,yint)", 
    "Reparametrization of the MPE as a function of $w, y_{int}$", 
)

Eq(P_{marg}, P_0*(-sqrt(w)*y + sqrt(w)*y_{int} + y)**2/(sqrt(w)*y_{int}**2))

In [64]:
FORMULAS.add(
    "MPWYiE", MPWYiE, 
    "Marginal Price Equation (w,yint)", 
    "Reparametrization of the MPE as a function of $w, y_{int}$", 
)

Eq(P_{marg}, P_0*(-sqrt(w)*y + sqrt(w)*y_{int} + y)**2/(sqrt(w)*y_{int}**2))

In [65]:
FORMULAS.add(
    "P0WYiE", P0WYiE, 
    "Constant Marginal Price Curve Equation", 
    "Establishes relationship $P_0=P_0(w)$ for which the marginal price remains const", 
)

Eq(P_0, P_{marg}*sqrt(w)*y_{int}**2/(-sqrt(w)*y + sqrt(w)*y_{int} + y)**2)

In [66]:
FORMULAS.add(
    "RXE", RXE, 
    "Ratio equation (x/x0)", 
    "Defines r as the ratio x/x0", 
)

Eq(x_0, r*x)

In [67]:
FORMULAS.add(
    "RXIE", RXIE, 
    "Inverse ratio equation (x0/x)", 
    "Defines rho as the ratio x0/x", 
)

Eq(x, \rho*x_0)

In [68]:
FORMULAS.add(
    "MPRE", MPRE, 
    "Marginal Price Equation (fixed x/x0)", 
    "this is the same as the Marginal Swap Equation, except that the LHS is recognized as the marginal price; x/x0 = r is fixed", 
)

Eq(P_{marg}, P_0*r**2/(\Gamma*(r - 1) - r)**2)

In [69]:
FORMULAS.add(
    "MPRIE", MPRIE, 
    "Marginal Price Equation (fixed x0/x)", 
    "this is the same as the Marginal Swap Equation, except that the LHS is recognized as the marginal price; x/x0 = r is fixed", 
)

Eq(P_{marg}, P_0/(\Gamma*(\rho - 1) + 1)**2)

### Definition equations and other relationships

In [70]:
FORMULAS.add(
    "QGDE", QGDE, 
    "Q Gamma Definition Equation", 
    "Gamma as a function of Q", 
)

Eq(Q, (1 - \Gamma)**2)

In [71]:
FORMULAS.add(
    "GQE", GQE, 
    "Gamma Q Equation", 
    "Gamma as a function of Q", 
)

Eq(\Gamma, 1 - sqrt(Q))

In [72]:
FORMULAS.add(
    "GFQE", GFQE, 
    "Gamma Q Formula Equation", 
    "an important relationship between functions of Gamma and Q", 
)

Eq((2 - \Gamma)/(1 - \Gamma), 1 + 1/sqrt(Q))

In [73]:
FORMULAS.add(
    "XiDE", XiDE, 
    "Xint Definition Equation", 
    "xint as function of x0 and Gamma", 
)

Eq(x_{int}, x_0*(2 - \Gamma)/(1 - \Gamma))

In [74]:
FORMULAS.add(
    "YiDE", YiDE, 
    "Yint Definition Equation", 
    "xint as function of y0 and Gamma", 
)

Eq(y_{int}, y_0*(2 - \Gamma)/(1 - \Gamma))

In [75]:
FORMULAS.add(
    "XiQE", XiQE, 
    "Xint Q Equation", 
    "xint as function of x0 and Q", 
)

Eq(x_{int}, x_0*(1 + 1/sqrt(Q)))

In [76]:
FORMULAS.add(
    "YiQE", YiQE, 
    "Yint Q Equation", 
    "yint as function of y0 and Q", 
)

Eq(y_{int}, y_0*(1 + 1/sqrt(Q)))

In [77]:
FORMULAS.add(
    "PxDE", PxDE, 
    "Px Equation", 
    "Px as function of P and Q", 
)

Eq(P_x, P_0*Q)

In [78]:
FORMULAS.add(
    "PyDE", PyDE, 
    "Py Equation", 
    "Py as function of P and Q", 
)

Eq(P_y, P_0/Q)

In [79]:
FORMULAS.add(
    "PXYE", PXYE, 
    "PXY Equation", 
    "P as a function of x0 and y0", 
)

Eq(P_0, y_0/x_0)

In [80]:
FORMULAS.add(
    "PXYiE", PXYiE, 
    "PXYi Equation", 
    "P as a function of xint and yint", 
)

Eq(P_0, y_{int}/x_{int})

In [81]:
FORMULAS.add(
    "QPE", QPE, 
    "QP Equation", 
    "Q as a function of Px and Py", 
)

Eq(Q, sqrt(P_y/P_x))

In [82]:
FORMULAS.add(
    "WDE", WDE, 
    "Range width definition equation", 
    "w as ratio Py/Px ", 
)

Eq(w, P_y/P_x)

In [83]:
FORMULAS.add(
    "WE", WE, 
    "Range width equation", 
    "Py/Px as a function of Q", 
)

Eq(w, Q**2)

In [84]:
FORMULAS.add(
    "PPE", PPE, 
    "PP Equation", 
    "P as a function of Px and Py", 
)

Eq(P_0, sqrt(P_x*P_y))

In [85]:
FORMULAS.add(
    "XiPE", XiPE, 
    "Xint from P", 
    "xint as function of Px and Py", 
)

Eq(x_{int}, y_{int}/sqrt(P_x*P_y))

In [86]:
FORMULAS.add(
    "YiPE", XiPE, 
    "yint from P", 
    "yint as function of Px and Py", 
)

Eq(x_{int}, y_{int}/sqrt(P_x*P_y))